In [33]:
################################### setting path ###################################
import sys
sys.path.append('../')
sys.path.append('../../')
################################### import libs ###################################
from  pytorchyolo import  models_split_tiny
# from pytorchyolo import models
import numpy as np
import time
import torch
import math
import os
from split_framework.split_framework_dynamic import SplitFramework
import tqdm
import numpy as np
import requests, pickle
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
from terminaltables import AsciiTable
from pytorchyolo.utils.utils import load_classes, ap_per_class, get_batch_statistics, xywh2xyxy
from pytorchyolo.utils.datasets import ListDataset
from pytorchyolo.utils.transforms import DEFAULT_TRANSFORMS
from algorithm.manager_full import Manager
from split_framework.gnb import GNB
import random
from scipy.interpolate import griddata
import pandas as pd
from torchinfo import summary

import warnings

import torch.ao.quantization.quantize_fx as qfx

In [30]:
cfg_path = "../../pytorchyolo/config/yolov3-tiny.cfg"
model_path = "../ckpt/vidVRD.pth"

split_layer=1


In [31]:
model = models_split_tiny.load_model(cfg_path, model_path)
model.set_split_layer(split_layer) # layer <7
model = model.eval()
summary(model, input_size=(1, 3, 416, 416))

Layer (type:depth-idx)                   Output Shape              Param #
Darknet                                  [1, 16, 208, 208]         --
├─ModuleList: 1-1                        --                        8,851,902
│    └─Sequential: 2-1                   [1, 16, 416, 416]         --
│    │    └─Conv2d: 3-1                  [1, 16, 416, 416]         432
│    │    └─BatchNorm2d: 3-2             [1, 16, 416, 416]         32
│    │    └─LeakyReLU: 3-3               [1, 16, 416, 416]         --
│    └─Sequential: 2-2                   [1, 16, 208, 208]         --
│    │    └─MaxPool2d: 3-4               [1, 16, 208, 208]         --
Total params: 8,852,366
Trainable params: 8,852,366
Non-trainable params: 0
Total mult-adds (M): 74.76
Input size (MB): 2.08
Forward/backward pass size (MB): 44.30
Params size (MB): 0.00
Estimated Total Size (MB): 46.38

In [34]:
torch.backends.quantized.engine = 'fbgemm'
qconfig_mapping = qfx.get_default_qconfig_mapping('fbgemm')

example_inputs = (torch.randn(1,3,416,416),)
prepared = qfx.prepare_fx(model, qconfig_mapping, example_inputs)

with torch.inference_mode():
    for imgs in calib_loader:
        prepared(imgs)

int8_model = qfx.convert_fx(prepared)

RuntimeError: quantized engine FBGEMM is not supported